In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis, skew

import os

In [ ]:
def load_data_csv(names : dict, path : str):
    """
    Carga archivos CSV en un directorio y los almacena como DataFrames de pandas.
    """
    dataframes = []
    dataframes_names = []
    for csv in os.listdir(path):
        if csv.endswith('.csv'):
            file_path = os.path.join(path, csv)
            if csv in names:
                df_name = names[csv]
                globals()[df_name] = pd.read_csv(file_path)
                dataframes.append(globals()[df_name])
                dataframes_names.append(df_name)
                print(f"DataFrame {df_name} cargado con éxito. ✅")
            else:
                print(f"DataFrame {csv} no cargado, no se encuentra en el diccionario. ")
    print("Nombres de los DataFrames cargados:", dataframes_names)
    return dataframes_names

In [ ]:
def get_df_size(df_names : list):
    """
    Devuelve un DataFrame con el tamaño de cada DataFrame cargado.
    """
    df_info = []
    for df_name in df_names:
        df = globals()[df_name]
        df_info.append({
            'name': df_name,
            'rows': df.shape[0],
            'columns': df.shape[1],
        })
    df_info = pd.DataFrame(df_info)
    df_info['size'] = df_info['rows'] * df_info['columns']
    df_info.sort_values(by='size', ascending=False, inplace=True)
    df_info.reset_index(drop=True, inplace=True)
    return df_info

In [ ]:
def get_df_dtypes(df_names: list, dataframes: list):
    """
    Devuelve un DataFrame con el conteo de tipos de datos de cada DataFrame cargado.
    """
    data_types_summary = []
    for df_name, df in zip(df_names, dataframes):
        data_types = df.dtypes.value_counts()
        summary = {"DataFrame": df_name}
        for dtype, count in data_types.items():
            summary[str(dtype)] = count
        data_types_summary.append(summary)
    data_types_summary_df = pd.DataFrame(data_types_summary).fillna(0)
    data_types_summary_df['total'] = data_types_summary_df.iloc[:, 1:].sum(axis=1)
    data_types_summary_df.sort_values(by='total', ascending=False, inplace=True)
    data_types_summary_df.reset_index(drop=True, inplace=True)
    return data_types_summary_df

In [ ]:
def get_df_columns(df_names: list, dataframes: list): 
    """
    Devuelve un DataFrame con los nombres de las columnas y su tipo de datos por DataFrame.
    """
    columns_summary = []
    for df_name, df in zip(df_names, dataframes):
        sorted_columns = sorted(df.columns)
        for column in sorted_columns:
            columns_summary.append({
                "DataFrame": df_name,
                "Column": column,
                "Type": str(df[column].dtype)
            })
    columns_summary_df = pd.DataFrame(columns_summary)
    columns_summary_df.sort_values(by='Column', ascending=True, inplace=True)
    columns_summary_df.reset_index(drop=True, inplace=True)
    return columns_summary_df

In [ ]:
def get_df_duplicates(df_names: list, dataframes: list):
    """
    Devuelve un DataFrame con el conteo de duplicados y su porcentaje por DataFrame.
    """
    results_duplicates = []
    for df_name, df in zip(df_names, dataframes):
        total_duplicates = df.duplicated().sum()
        total_rows = df.shape[0]
        percentage_duplicates = round((total_duplicates / total_rows) * 100, 2) if total_rows > 0 else 0.0
        results_duplicates.append({
            "DataFrame": df_name,
            "total_duplicates": total_duplicates,
            "%_duplicates": percentage_duplicates
        })
    duplicates_summary = pd.DataFrame(results_duplicates)
    duplicates_summary.sort_values(by="%_duplicates", ascending=False, inplace=True)
    duplicates_summary.reset_index(drop=True, inplace=True)
    return duplicates_summary

In [ ]:
def detect_outliers(df, umbral_porcentaje=1.0):
    """
    Detecta outliers en variables numéricas utilizando el método IQR (Interquartile Range).
    Los outliers se definen como valores que están por debajo de Q1 - 1.5 * IQR o por encima de Q3 + 1.5 * IQR.
    Se imprime un resumen de las variables con outliers y se generan gráficos para cada variable afectada.
    """
    print("🔎 Detección de variables numéricas con outliers (IQR)")
    print("=" * 60)
    resumen = []
    numeric_columns = df.select_dtypes(include=['float64', 'int64'])
    for column in numeric_columns.columns:
        serie = df[column].dropna()
        if len(serie) == 0:
            continue
        Q1 = serie.quantile(0.25)
        Q3 = serie.quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = serie[(serie < lower) | (serie > upper)]
        porcentaje_outliers = 100 * len(outliers) / len(serie)
        if porcentaje_outliers >= umbral_porcentaje:
            print(f"📌 Variable: '{column}'")
            print("-" * 40)
            print(f"Total de datos: {len(serie)}")
            print(f"Outliers detectados: {len(outliers)}")
            print(f"Porcentaje de outliers: {porcentaje_outliers:.2f}%")
            print(f"Rango IQR: Q1 = {Q1:.2f}, Q3 = {Q3:.2f}, IQR = {IQR:.2f}")
            print(f"Valores extremos: min = {serie.min()}, max = {serie.max()}")
            print()
            plt.figure(figsize=(12, 5))
            plt.subplot(1, 2, 1)
            sns.histplot(serie, kde=True, bins=30, color='steelblue')
            plt.axvline(lower, color='red', linestyle='--', label='Límite inferior')
            plt.axvline(upper, color='red', linestyle='--', label='Límite superior')
            plt.title(f'Distribución de {column}')
            plt.legend()
            plt.subplot(1, 2, 2)
            sns.boxplot(x=serie, color='skyblue')
            plt.axvline(lower, color='red', linestyle='--')
            plt.axvline(upper, color='red', linestyle='--')
            plt.title(f'Boxplot de {column}')
            plt.tight_layout()
            plt.show()
            print("=" * 60 + "\n")
            resumen.append({
                'Variable': column,
                'Total valores': len(serie),
                'Outliers': len(outliers),
                '% Outliers': round(porcentaje_outliers, 2),
                'Q1': Q1,
                'Q3': Q3,
                'IQR': IQR,
                'Min': serie.min(),
                'Max': serie.max()
            })
    resultado = pd.DataFrame(resumen).sort_values(by='% Outliers', ascending=False)
    resultado = resultado.reset_index(drop=True)
    if resultado.empty:
        print("✅ No se detectaron variables con outliers por encima del umbral.")
    else:
        print("📈 Resumen de variables con outliers:")
    return resultado


In [ ]:
def identify_null_values(df):
    """
    Identifica y resume los valores nulos en un DataFrame.
    """
    total_cells = df.size
    total_rows = df.shape[0]
    total_nulls = df.isnull().sum().sum()
    porcentage_total = round((total_nulls / total_cells) * 100, 2)
    nulls_by_column = df.isnull().sum()
    porcentage_by_columna = round((nulls_by_column / total_rows) * 100, 2)
    types = df.dtypes
    resumen = pd.concat([nulls_by_column, porcentage_by_columna, types], axis=1)
    resumen.columns = ['total_nulls', '%_nulos', 'type']
    resumen = resumen[resumen['total_nulls'] > 0].sort_values(by='total_nulls', ascending=False)
    print("🔍 Resumen general de valores nulos:")
    print(f"Total de celdas del DataFrame: {total_cells}")
    print(f"Total de valores nulos: {total_nulls}")
    print(f"Porcentaje de valores nulos sobre el total: {porcentage_total}%")
    print(f"Número de columnas con al menos un valor nulo: {resumen.shape[0]}")
    print("\n📊 Detalle por columna:\n")
    return resumen

In [ ]:
def analyze_null_correlations(df, df_name='DataFrame'):
    """
    Analiza la correlación entre columnas con valores nulos en un DataFrame.
    Genera una matriz de correlación y un gráfico de calor para visualizar las correlaciones.
    """
    null_columns = df.columns[df.isnull().any()]
    if len(null_columns) == 0:
        print(f"✅ El DataFrame '{df_name}' no contiene valores nulos.")
        return None
    null_binary = df[null_columns].isnull().astype(int)
    null_corr = null_binary.corr()
    print(f"🔍 Matriz de correlación de valores nulos para: {df_name}\n")
    print(null_corr)
    mask = np.triu(np.ones_like(null_corr, dtype=bool))
    filtered_corr = null_corr.where(np.abs(null_corr) > 0.01)
    plt.figure(figsize=(50, 30))
    sns.heatmap(filtered_corr, mask=mask, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f", cbar=True)
    plt.title(f'📊 Correlación entre columnas con nulos: {df_name}', fontsize=14)
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    return null_corr

In [ ]:
def univariate_analysis_numeric(df):
    """
    Realiza un análisis univariado de las variables numéricas en un DataFrame.
    Genera estadísticas descriptivas, gráficos de distribución y boxplots.
    Calcula la asimetría y curtosis de cada variable numérica.
    """
    numeric_columns = df.select_dtypes(include=['float64', 'int64'])
    for column in numeric_columns.columns:
        print(f"🔍 Análisis univariado para la columna '{column}':")
        print("-" * 40)
        serie = df[column].dropna()
        print("📊 Estadísticas descriptivas:")
        print(serie.describe())
        print()
        skewness = skew(serie)
        kurt = kurtosis(serie)
        print(f"Asimetría (Skewness): {skewness:.2f}")
        print(f"Curtosis (Kurtosis): {kurt:.2f}")
        if abs(skewness) < 0.5:
            skewness_desc = "Distribución aproximadamente simétrica"
        elif skewness > 0.5:
            skewness_desc = "Distribución sesgada a la derecha (positiva)"
        else:
            skewness_desc = "Distribución sesgada a la izquierda (negativa)"
        if kurt > 3:
            kurt_desc = "Distribución leptocúrtica (colas más largas)"
        elif kurt < 3:
            kurt_desc = "Distribución platicúrtica (colas más cortas)"
        else:
            kurt_desc = "Distribución mesocúrtica (similar a la normal)"
        print(f"📌 Interpretación de la asimetría: {skewness_desc}")
        print(f"📌 Interpretación de la curtosis: {kurt_desc}")
        print()
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        sns.histplot(serie, kde=True, bins=30, color='steelblue')
        plt.title(f'Distribución de {column}')
        plt.subplot(1, 2, 2)
        sns.boxplot(x=serie, color='skyblue')
        plt.title(f'Boxplot de {column}')
        plt.tight_layout()
        plt.show()
        print("\n" + "="*60 + "\n")

In [ ]:
def univariate_analysis_categorical(df):
    """
    Realiza un análisis univariado de las variables categóricas en un DataFrame.
    Genera tablas de frecuencia y gráficos de barras para cada variable categórica.
    """
    categorical_columns = df.select_dtypes(include=['object', 'category'])
    for column in categorical_columns.columns:
        print(f"🔍 Análisis univariado para la columna '{column}':")
        print("-" * 40)
        serie = df[column]
        value_counts = serie.value_counts(dropna=False)
        percentages = round((value_counts / len(serie)) * 100, 1)
        freq_table = pd.DataFrame({
            'Frecuencia': value_counts,
            'Porcentaje (%)': percentages
        })
        print("📊 Frecuencia y porcentaje de cada categoría:")
        print(freq_table)
        print()
        unique_categories = serie.nunique(dropna=False)
        if unique_categories > 50:
            print(f"⚠️ La columna '{column}' tiene muchas categorías ({unique_categories}). Se omite la gráfica.\n")
            print("=" * 60 + "\n")
            continue
        plt.figure(figsize=(15, 5))
        sns.barplot(
            x=value_counts.index.astype(str), 
            y=value_counts.values, 
            palette="viridis"
        )
        plt.title(f'📈 Frecuencia de categorías en {column}')
        plt.xlabel('Categorías')
        plt.ylabel('Frecuencia')
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.show()
        print("\n" + "=" * 60 + "\n")